# Download 5-minutes data from yahoofinance

In [ ]:
# import Python libraries
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
from datetime import date
from datetime import timedelta
import os
import time

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Today's date
today = date.today()
# dd/mm/YY
d1 = today.strftime("%m_%d_%Y")

# 40 FTSEMIB companies
companies_dict={
                "A2A.MI":"A2A","AMP.MI":"AMPLIFON","ATL.MI":"ATLANTIA",
                "AZM.MI":"AZIMUT","BGN.MI":"BANCA GENERALI", "BMED.MI":"BANCA MEDIOLANUM",
                "BAMI.MI":"BANCO BPM","BPE.MI":"BPER BANCA",
                "BZU.MI":"BUZZI UNICEM","CPR.MI":"CAMPARI","CNHI.MI":"CNH INDUSTRIAL","DIA.MI":"DIASORIN",
                "ENEL.MI":"ENEL","ENI.MI":"ENI","EXO.MI":"EXOR",
                "RACE.MI":"FERRARI","FBK.MI":"FINECO","G.MI":"GENERALI ASSICURAZIONI",
                "HER.MI":"HERA", "IP.MI":"INTERPUMP GROUP",
                "ISP.MI":"INTESA SAN PAOLO", "INW.MI":"INWIT", "IG.MI":"ITALGAS",
                "LDO.MI":"LEONARDO","MB.MI":"MEDIOBANCA","MONC.MI":"MONCLER","NEXI.MI":"NEXI", 
                "PIRC.MI":"PIRELLI","PST.MI":"POSTE ITALIANE",
                "PRY.MI":"PRYSMIAN","REC.MI":"RECORDATI","SPM.MI":"SAIPEM","SRG.MI":"SNAM",
                "STLA.MI":"STELLANTIS", "STM.MI":"STM","TIT.MI":"TELECOM",
                "TEN.MI":"TENARIS","TRN.MI":"TERNA", "UCG.MI":"UNICREDIT","UNI.MI":"UNIPOL"
}


In [ ]:
# we need to have every 5-minutes data ...so I create a list 

time_ = '09:00'
timeList = '00:05'
count = 0
hours_list = []
mysum = 0
while str(mysum) != '17:30:00':
    if count<1:
        td = timedelta(hours=int(time_.split(':')[0]), minutes=int(time_.split(':')[1]))
        d = timedelta(hours=int(timeList.split(':')[0]), minutes=int(timeList.split(':')[1]))
        mysum = td + d
    else:
        d = timedelta(hours=int(timeList.split(':')[0]), minutes=int(timeList.split(':')[1]))
        mysum = td + d
    td = mysum
    count+=1
    hours_list.append('{}:{}'.format(str(mysum).split(':')[0], str(mysum).split(':')[1]))

In [ ]:
def download_stock_prices(ticker, name, folder):

    df = yf.download(tickers=ticker, period='60d', interval="5m")
    # we build 'TIME' and 'DATE' variables
    df['TIME'] =[n for n in df.reset_index()['Datetime'].dt.time]
    df['DATE'] = [str(n).split(' ')[0] for n in df.reset_index()['Datetime']]
    
    # empty list; we set the date in a specific form
    lista = []
    for h in df['DATE']:
        l = h.split('-')
        l2 = [l[2],l[1],l[0]]
        lista.append('/'.join(l2))
    
    df['DATE'] = lista
    # convert DATE into datetime
    df['DATE'] = pd.to_datetime(df['DATE'], format='%d/%m/%Y')
    # we set the 'TIME' into 'hours:minutes'
    lis = []
    for j in df['TIME']:
        j = str(j)
        j = j.split(':')
        mm = [j[0], j[1]]
        m = ':'.join(mm)
        lis.append(m)
    df['TIME'] = lis
    
    #we have to build a dataframe
    dataframes = []
    d_orari = pd.DataFrame({'TIME' : hours_list})
    for h in df['DATE'].unique():
        d_2 = df[df['DATE'] == h]
        # yahoo finance data starts at 9:00 so we need to move 5 minutes ahead
        d_2['TIME'] = pd.to_datetime(d_2['TIME'], format='%H:%M')
        d_2['TIME'] = d_2['TIME'] + timedelta(0,300)
    
    
        lis = []
        for j in d_2['TIME']:
            j = str(j)
            j = j.split(' ')
            mm = j[1]
            j = mm.split(':')
            mmm = [j[0], j[1]]
            m = ':'.join(mmm)
            lis.append(m)
        
        d_2['TIME'] = lis
        
        #merge with 'only hours' dataset (we want every work hours to understand if there are 5-minutes without buy/sell)
        d_2 = d_orari.merge(d_2, on='TIME', how='left')
        d_2['DATE'] = h
        #d_2 = d_2.fillna(method='ffill')
        #d['TIME'] = pd.to_datetime(d['TIME'], format='%H:%M').dt.time
        d_2['DATE_TIME'] = d_2['DATE'].astype(str) + ' ' + d_2['TIME'].astype(str)
        d_2 = d_2.sort_values('DATE_TIME')
        d_2['day_of_week'] = d_2['DATE'].dt.day_name()
        d_2['num_day_of_of_week'] = d_2['DATE'].dt.dayofweek
        d_2['weeks'] = d_2['DATE'].dt.week
        d_2['year'] = d_2['DATE'].dt.year
        d_2['month'] = d_2['DATE'].dt.month
        dataframes.append(d_2)
    reading_concatenation = pd.concat(dataframes, ignore_index=True)
    reading_concatenation.index = reading_concatenation['DATE_TIME']
    reading_concatenation = reading_concatenation.rename(columns={'Open': 'OPEN', 'High':'HIGH', 'Low':'LOW','Close' :'CLOSE', 'Adj Close': 'ADJ CLOSE', 'Volume': 'VOLUME'})
    new_df = pd.DataFrame(reading_concatenation, columns=['TIME', 'DATE', 'OPEN', 'HIGH', 'LOW', 'CLOSE','ADJ CLOSE', 'VOLUME', 'DATE_TIME', 'day_of_week', 'num_day_of_of_week', 'weeks', 'year', 'month'])

    dirName = folder + '\{}'.format(name)
    print(dirName)
    if not os.path.exists(dirName):
        os.makedirs(dirName)
        print("Directory " , dirName ,  " Created ")
    else:    
        print("Directory " , dirName ,  " already exists")
    os.chdir(r'{}'.format(dirName))
    new_df.to_csv('1{}_until_{}.csv'.format(name, d1), index = False)
    
    return new_df

In [ ]:
# This function adds a file csv with all data in ourfolder

def read_concatenation(name, folder):
    dataframes = []
    
    dirName = folder + '\{}'.format(name)
    if not os.path.exists(dirName):
        print("ERROR " , dirName ,  " doesn't exist!!!!!")
    else:    
        print("Directory " , dirName ,  " already exists")
    directory = r'{}'.format(dirName)
    length = len(os.listdir(directory))
    if length < 2:
        for filename in os.listdir(directory):
                if filename.startswith("1{}".format(name)) and filename.endswith(".csv"):
                    d = pd.read_csv(directory+'/'+filename)
                    dataframes.append(d)
                else:
                    print('Skip: {}'.format(filename))
                          
        reading_concatenation = pd.concat(dataframes, ignore_index=True)
        cleaned_dataset = reading_concatenation.drop_duplicates(subset ="DATE_TIME", keep = 'first')
        os.chdir(r'{}'.format(dirName))
        cleaned_dataset.to_csv('Complete_{}.csv'.format(name), index = False)
    else:
        print('Più elementi in: ', name)
        elements = os.listdir(r'{}'.format(directory))[-2:length]
        elements.reverse()
        for filename in elements:
                if filename.endswith(".csv"):
                    d = pd.read_csv(directory+'/'+filename)
                    dataframes.append(d)
                else:
                    print('Skip: {}'.format(filename))
                          
        reading_concatenation = pd.concat(dataframes, ignore_index=True)
        cleaned_dataset = reading_concatenation.drop_duplicates(subset ="DATE_TIME", keep = 'last')
        os.chdir(r'{}'.format(directory))
        cleaned_dataset.to_csv('Complete_{}.csv'.format(name), index = False)

In [ ]:
input_dir = input('Select your directory: ')
for key, value in companies_dict.items():
    download_stock_prices(key, value, input_dir)
for name_company in companies_dict.values():
    read_concatenation(name_company, input_dir)
    print('{}: Done'.format(name_company))